# XBGclassifier

In [1]:
from xgboost import XGBClassifier
#from imblearn.over_sampling import SMOTE
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [2]:
df1=pd.read_csv('df4.csv', encoding='ISO-8859-1')
df1.head()

,cleaned_text,sentiment
0,what said,neutral
1,plus youve added commercials to the experience...,positive
2,i didnt today must mean i need to take another...,neutral
3,its really aggressive to blast obnoxious enter...,NaN
4,and its a really big bad thing about it,negative


## turn unlabled to -1 *rules of selfTrainClassifier

In [3]:
X = df1["cleaned_text"]
y = df1["sentiment"]

y[y.isna()] = -1
print(y.isna().sum())

0


In [4]:
y = y.map({
    -1:-1,
    "negative":0,
    "neutral": 1,
    "positive": 2
})

In [5]:
X_labeled = X[y != -1]
y_labeled = y[y != -1]

X_unlabeled = X[y == -1]
y_unlabeled = y[y == -1]

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X_labeled,y_labeled,test_size=0.2)
X_train = pd.concat([X_train,X_unlabeled])
y_train = pd.concat([y_train,y_unlabeled])

## Applying XGBclassifier

In [7]:
model6 = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("model", SelfTrainingClassifier(XGBClassifier()))
])

In [8]:
# Train the model
model6.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('model',
                 SelfTrainingClassifier(base_estimator=XGBClassifier(base_score=None,
                                                                     booster=None,
                                                                     callbacks=None,
                                                                     colsample_bylevel=None,
                                                                     colsample_bynode=None,
                                                                     colsample_bytree=None,
                                                                     device=None,
                                                                     early_stopping_rounds=None,
                                                                     enable_categorical=False,
                                                                     eval_metric=None,
                                                                     feature_types=None,
                                                                     gamma=None,
                                                                     grow_policy=None,
                                                                     importance_type=None,
                                                                     interaction_constraints=None,
                                                                     learning_rate=None,
                                                                     max_bin=None,
                                                                     max_cat_threshold=None,
                                                                     max_cat_to_onehot=None,
                                                                     max_delta_step=None,
                                                                     max_depth=None,
                                                                     max_leaves=None,
                                                                     min_child_weight=None,
                                                                     missing=nan,
                                                                     monotone_constraints=None,
                                                                     multi_strategy=None,
                                                                     n_estimators=None,
                                                                     n_jobs=None,
                                                                     num_parallel_tree=None,
                                                                     random_state=None, ...)))])

In [9]:
# Make predictions on the test data
y_pred = model6.predict(X_test)

In [10]:
# Compute F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # You can change the average parameter based on your needs
print("F1 Score is : ", f1)

F1 Score is :  0.74663996860824


In [11]:
# Print the accuracy score
print("Accuracy Score is : ", accuracy_score(y_test, y_pred))

Accuracy Score is :  0.7615253272623791


In [12]:
# Print the classification report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.92      0.84      1080
           1       0.65      0.44      0.52       363
           2       0.81      0.59      0.68       314

    accuracy                           0.76      1757
   macro avg       0.74      0.65      0.68      1757
weighted avg       0.75      0.76      0.75      1757



## Applying XGBclassifier using Over-sampling with SMOTE

In [13]:
model6 = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("smote", SMOTE()),  # Over-sampling using SMOTE
    ("model", SelfTrainingClassifier(XGBClassifier()))
])

In [14]:
# Train the model
model6.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE()),
                ('model',
                 SelfTrainingClassifier(base_estimator=XGBClassifier(base_score=None,
                                                                     booster=None,
                                                                     callbacks=None,
                                                                     colsample_bylevel=None,
                                                                     colsample_bynode=None,
                                                                     colsample_bytree=None,
                                                                     device=None,
                                                                     early_stopping_rounds=None,
                                                                     enable_categorical=False,
                                                                     eval_metric=None,
                                                                     feature_types=None,
                                                                     gamma=None,
                                                                     grow_policy=None,
                                                                     importance_type=None,
                                                                     interaction_constraints=None,
                                                                     learning_rate=None,
                                                                     max_bin=None,
                                                                     max_cat_threshold=None,
                                                                     max_cat_to_onehot=None,
                                                                     max_delta_step=None,
                                                                     max_depth=None,
                                                                     max_leaves=None,
                                                                     min_child_weight=None,
                                                                     missing=nan,
                                                                     monotone_constraints=None,
                                                                     multi_strategy=None,
                                                                     n_estimators=None,
                                                                     n_jobs=None,
                                                                     num_parallel_tree=None,
                                                                     random_state=None, ...)))])

In [15]:
# Make predictions on the test data
y_pred = model6.predict(X_test)

In [16]:
# Compute F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # You can change the average parameter based on your needs
print("F1 Score is : ", f1)

F1 Score is :  0.7647544645624713


In [17]:
# Print the accuracy score
print("Accuracy Score is : ", accuracy_score(y_test, y_pred))

Accuracy Score is :  0.7683551508252704


In [18]:
# Print the classification report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.88      0.85      1080
           1       0.60      0.59      0.59       363
           2       0.78      0.61      0.68       314

    accuracy                           0.77      1757
   macro avg       0.73      0.69      0.71      1757
weighted avg       0.77      0.77      0.76      1757

